In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import time
import pyaudio
import wave
import speech_recognition as sr
from transformers import pipeline
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [15]:
class GoogleSheetsIntegration:
    def __init__(self, credentials_path):
        scope = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        self.creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        self.client = gspread.authorize(self.creds)

    def append_row(self, spreadsheet_id, data):
        try:
            sheet = self.client.open_by_key(spreadsheet_id).sheet1
            sheet.append_row(data)
        except Exception as e:
            print(f"Error appending row: {e}")

In [16]:
def record_audio_chunk(chunk_duration=4, file_name="buyer_audio.wav"):
    """Records a chunk of audio for the specified duration."""
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    rate = 44100

    p = pyaudio.PyAudio()
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)

    print("Listening for the buyer's input...")
    frames = []

    for _ in range(0, int(rate / chunk * chunk_duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    return file_name

In [17]:
def transcribe_audio(file_name="buyer_audio.wav"):
    """Transcribes audio to text using Google Speech Recognition."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_name) as source:
        print("Processing audio transcription...")
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Transcription: {text}")
            return text
        except sr.UnknownValueError:
            print("No clear speech detected in the last chunk.")
            return ""
        except sr.RequestError as e:
            print(f"Error in request: {e}")
            return ""

In [30]:
def analyze_sentiment(text):
    """Analyzes the sentiment of the given text."""
    sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    
    # Define a mapping from model labels to human-readable labels
    label_mapping = {
        "LABEL_0": "Negative",
        "LABEL_1": "Neutral",
        "LABEL_2": "Positive"
    }
    
    sentiment_result = sentiment_pipeline(text)[0]
    human_readable_label = label_mapping[sentiment_result["label"]]
    
    return human_readable_label
def analyze_intent(text):
    """Analyzes the intent of the given text."""
    intent_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    candidate_labels =  ["Greeting", "Inquiry", "Giving Feedback", "Willing to Buy", "Not Interested"]
    result = intent_model(text, candidate_labels)
    return result['labels'][0]

In [33]:
def generate_negotiation_terms(sentiment, intent, context):
    if intent == "Greeting":
        return "Thank you for reaching out! How can I assist you today?"
    elif intent == "Inquiry":
        if "price" in context or "discount" in context:
            if sentiment == "Neutral":
                return "We can explore potential discounts or payment plans that suit your preferences. How would you like to proceed?"
            elif sentiment == "Positive":
                return "We’d love to work out a price that meets your expectations. Let’s discuss further!"
            else:
                return "While our pricing is competitive, we can offer additional perks or flexible payment options to make this work for you."
        else:
            return "Let me provide more details about the product. Please feel free to ask any specific questions you have."
    elif intent == "Giving Feedback":
        if sentiment == "Positive":
            return "Thank you for your positive feedback! We're here to help if you have any further questions or concerns."
        else:
            return "We value your feedback and will work on addressing your concerns. Please let us know if there’s anything specific we can do to assist you."
    elif intent == "Willing to Buy":
        if sentiment == "Positive":
            return "That's wonderful to hear! Let’s finalize the details and get everything ready for you."
        elif sentiment == "Neutral":
            return "We’re glad to hear you’re considering it! Let’s discuss the next steps and how we can make this process easier for you."
        else:
            return "We appreciate your interest. Let’s address any remaining concerns to move forward confidently."
    elif intent == "Not Interested":
        if sentiment == "Negative":
            return "We understand that this might not meet your expectations. If there’s anything else you’re looking for, let us know—we’re happy to assist!"
        else:
            return "Thank you for letting us know. If you change your mind or have any other requirements, feel free to reach out!"
    else:
        return "Thank you for sharing your thoughts. How can we assist you further?"

In [34]:
 def main():
    """Main workflow for continuous buyer interaction."""
    credentials_path = "credentials.json"  # Path to your Google credentials file
    spreadsheet_id = "1I3lanT_YNQ9GELA18oXhDLY2msww5uwOvHlx92uync8"  # Replace with your spreadsheet ID

    sheets_integration = GoogleSheetsIntegration(credentials_path)

    print("Starting continuous buyer interaction...")

    while True:
        # Record a chunk of audio
        audio_file = record_audio_chunk()

        # Transcribe the audio
        transcribed_text = transcribe_audio(audio_file)
        if not transcribed_text:
            print("Waiting for the next buyer input...\n")
            time.sleep(4)  # Pause to wait for the buyer to speak
            continue

        # Analyze sentiment and intent
        sentiment = analyze_sentiment(transcribed_text)
        intent = analyze_intent(transcribed_text)

        # Generate negotiation terms
        negotiation_terms = generate_negotiation_terms(sentiment, intent, transcribed_text)

        # Log results in Google Sheets
        sheets_integration.append_row(
            spreadsheet_id,
            [
                time.strftime("%Y-%m-%d %H:%M:%S"),  # Timestamp
                transcribed_text,
                sentiment,
                intent,
                negotiation_terms
            ]
        )

        # Print interaction details
        print("\nBuyer Transcription:", transcribed_text)
        print("Sentiment:", sentiment)
        print("Intent:", intent)
        print("Negotiation Terms:", negotiation_terms)
        print("Interaction logged in Google Sheets.")
        print("Waiting for the next interaction...\n")

        time.sleep(4)  # Wait for the next input


if __name__ == "__main__":
    main()

Starting continuous buyer interaction...
Listening for the buyer's input...
Processing audio transcription...
Transcription: do you offer any discounts

Buyer Transcription: do you offer any discounts
Sentiment: Neutral
Intent: Inquiry
Negotiation Terms: We can explore potential discounts or payment plans that suit your preferences. How would you like to proceed?
Interaction logged in Google Sheets.
Waiting for the next interaction...



KeyboardInterrupt: 